In [3]:
import pandas as pd
import numpy as np

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.3f}'.format)

In [11]:
OUTPUT_DIR="output_experimental"
OUTPUT_TIME="time"

EXECUTIONS = 50
CITIES = [13, 14, 15]
THREADS = [1, 2, 3, 6, 9, 12]
ITERATIONS = 10

In [5]:
def geometric_mean_of_mean_of_instances(mean_of_instances):
    geometric_mean = mean_of_instances.prod()**(1.0/EXECUTIONS)
    return geometric_mean

def std_mean_of_mean_of_instances(mean_of_instances):
    # std_mean = np.std(mean_of_instances)/(EXECUTIONS**2)
    std_mean = np.std(mean_of_instances)
    return std_mean

def extract_time_stats_from_file(file_name):
    file = open(file_name)
    data = []
    for exec in range(1, EXECUTIONS + 1):
        data.append(float(file.readline().split(" ")[1]))
    np_array = np.array(data)
    file.close()
    return np.sum(np_array), np_array


In [13]:
df_time = pd.DataFrame(columns = ["Version", "Cities", "Threads", "Instance_mean", "Instance_std", "Instance_std_%(x100)", "Sum_mean", "Sum_std", "Sum_std_%(x100)"])

for city in list(CITIES):
    sum_array, array_of_instance_array = [], []
    for iteration in range(1,ITERATIONS + 1):
        file_name = OUTPUT_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        
        sum, instance_array = extract_time_stats_from_file(file_name)
        
        array_of_instance_array.append(instance_array)
        sum_array.append(sum)
    
    mean_of_instances = np.mean(array_of_instance_array, axis=0)
    
    geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
    std_mean = std_mean_of_mean_of_instances(mean_of_instances)
    list_row = ["Sequential", city, "1", geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
    df_time.loc[len(df_time)] = list_row
            
    for thread in list(THREADS):
        sum_array, array_of_instance_array = [], []
        for iteration in range(1,ITERATIONS + 1):
            file_name = OUTPUT_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(thread) + "_threads_result.out"
            
            sum, instance_array = extract_time_stats_from_file(file_name)

            array_of_instance_array.append(instance_array)
            sum_array.append(sum)

        mean_of_instances = np.mean(array_of_instance_array, axis=0)
        
        geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
        std_mean = std_mean_of_mean_of_instances(mean_of_instances)
        list_row = ["Parallel", city, thread, geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
        df_time.loc[len(df_time)] = list_row
df_time

,Version,Cities,Threads,Instance_mean,Instance_std,Instance_std_%(x100),Sum_mean,Sum_std,Sum_std_%(x100)
0,Sequential,13,1,0.382,0.397,103.981,24.774,0.187,0.757
1,Parallel,13,1,0.543,0.515,94.748,34.439,0.075,0.217
2,Parallel,13,2,0.218,0.217,99.918,13.918,0.018,0.133
3,Parallel,13,3,0.145,0.145,100.062,9.271,0.028,0.304
4,Parallel,13,6,0.074,0.073,99.308,4.716,0.010,0.216
5,Parallel,13,9,0.060,0.058,96.694,3.812,0.010,0.273
6,Parallel,13,12,0.052,0.049,94.485,3.268,0.007,0.204
7,Sequential,14,1,1.539,3.838,249.376,129.097,0.391,0.303
8,Parallel,14,1,2.397,4.984,207.937,185.243,0.248,0.134
9,Parallel,14,2,0.859,2.078,241.978,71.255,0.123,0.173


Speedup and Efficiency

In [14]:
df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Threads", "Instance_speedup", "Instance_efficiency", "Sum_speedup", "Sum_efficiency"])
# df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Threads", "Sum Speedup", "Sum Efficiency"])

df_sequential = df_time[df_time['Version'].isin(["Sequential"])]
df_parallel = df_time[df_time['Version'].isin(["Parallel"])]

for city in list(CITIES):
    df_seq = df_sequential[df_sequential["Cities"].isin([city])]
    for thread in list(THREADS):
        df_par = df_parallel[df_parallel["Cities"].isin([city]) & df_parallel["Threads"].isin([thread])]
        speedup_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()
        efficiency_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()/float(thread)
        speedup_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()
        efficiency_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()/float(thread)

        list_row = [city, thread, speedup_instance, efficiency_instance, speedup_sum, efficiency_sum]
        # list_row = [city, thread, speedup_sum, efficiency_sum]
        df_speedup_efficiency.loc[len(df_speedup_efficiency)] = list_row

df_speedup_efficiency["Threads"] = df_speedup_efficiency["Threads"].astype(int)
df_speedup_efficiency["Cities"] = df_speedup_efficiency["Cities"].astype(int)
df_speedup_efficiency

,Cities,Threads,Instance_speedup,Instance_efficiency,Sum_speedup,Sum_efficiency
0,13,1,0.703,0.703,0.719,0.719
1,13,2,1.755,0.878,1.780,0.890
2,13,3,2.637,0.879,2.672,0.891
3,13,6,5.167,0.861,5.253,0.875
4,13,9,6.345,0.705,6.499,0.722
5,13,12,7.376,0.615,7.581,0.632
6,14,1,0.642,0.642,0.697,0.697
7,14,2,1.792,0.896,1.812,0.906
8,14,3,2.707,0.902,2.743,0.914
9,14,6,5.315,0.886,5.387,0.898


Output 1

In [15]:
# translate to portuguese
df_time.loc[df_time["Version"] == "Parallel", "Version"] = "Paralelo"
df_time.loc[df_time["Version"] == "Sequential", "Version"] = "Sequencial"
res = df_time.rename(columns={
    "Version": "Versão",
    "Cities": "# Cidades",
    "Instance_mean": "Média geométrica instância",
    "Instance_std": "Desvio padrão instância",
    "Instance_std_%(x100)": "% Desvio padrão instância (x100)",
    "Sum_mean": "Média aritmética tempo total",
    "Sum_std": "Desvio padrão tempo total",
    "Sum_std_%(x100)": "% Desvio padrão tempo total (x100)"
})
res

,Versão,# Cidades,Threads,Média geométrica instância,Desvio padrão instância,% Desvio padrão instância (x100),Média aritmética tempo total,Desvio padrão tempo total,% Desvio padrão tempo total (x100)
0,Sequencial,13,1,0.382,0.397,103.981,24.774,0.187,0.757
1,Paralelo,13,1,0.543,0.515,94.748,34.439,0.075,0.217
2,Paralelo,13,2,0.218,0.217,99.918,13.918,0.018,0.133
3,Paralelo,13,3,0.145,0.145,100.062,9.271,0.028,0.304
4,Paralelo,13,6,0.074,0.073,99.308,4.716,0.010,0.216
5,Paralelo,13,9,0.060,0.058,96.694,3.812,0.010,0.273
6,Paralelo,13,12,0.052,0.049,94.485,3.268,0.007,0.204
7,Sequencial,14,1,1.539,3.838,249.376,129.097,0.391,0.303
8,Paralelo,14,1,2.397,4.984,207.937,185.243,0.248,0.134
9,Paralelo,14,2,0.859,2.078,241.978,71.255,0.123,0.173


In [16]:
res1 = res.pivot(index=["Threads", "Versão"], columns="# Cidades", values=["Média geométrica instância", "Desvio padrão instância", "% Desvio padrão instância (x100)"])
res2 = res.pivot(index=["Threads", "Versão"], columns="# Cidades", values=["Média aritmética tempo total", "Desvio padrão tempo total", "% Desvio padrão tempo total (x100)"])
res1 = res1.sort_values(by='Versão', ascending=False)
res2 = res2.sort_values(by='Versão', ascending=False)
res1

Média geométrica instância               \
# Cidades                                  13    14     15   
Threads Versão                                               
1       Sequencial                      0.382 1.539  7.627   
        Paralelo                        0.543 2.397 11.162   
2       Paralelo                        0.218 0.859  4.204   
3       Paralelo                        0.145 0.569  2.767   
6       Paralelo                        0.074 0.290  1.405   
9       Paralelo                        0.060 0.237  1.160   
12      Paralelo                        0.052 0.204  1.013   

                   Desvio padrão instância               \
# Cidades                               13    14     15   
Threads Versão                                            
1       Sequencial                   0.397 3.838  9.265   
        Paralelo                     0.515 4.984 13.572   
2       Paralelo                     0.217 2.078  5.060   
3       Paralelo                     0.145 1.377  3.322   
6       Paralelo                     0.073 0.704  1.694   
9       Paralelo                     0.058 0.567  1.385   
12      Paralelo                     0.049 0.480  1.201   

                   % Desvio padrão instância (x100)                  
# Cidades                                        13      14      15  
Threads Versão                                                       
1       Sequencial                          103.981 249.376 121.476  
        Paralelo                             94.748 207.937 121.594  
2       Paralelo                             99.918 241.978 120.371  
3       Paralelo                            100.062 242.231 120.045  
6       Paralelo                             99.308 243.127 120.561  
9       Paralelo                             96.694 239.159 119.329  
12      Paralelo                             94.485 234.643 118.598

In [17]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [18]:
res1

In [19]:
res2

Output 2

In [20]:
# translate to portuguese
res = df_speedup_efficiency.rename(columns={"Cities": "# Cidades", "Instance_speedup": "Speedup instância", "Instance_efficiency": "Eficiência instância", "Sum_speedup": "Speedup tempo total", "Sum_efficiency": "Eficiência tempo total"})
res

,# Cidades,Threads,Speedup instância,Eficiência instância,Speedup tempo total,Eficiência tempo total
0,13,1,0.703,0.703,0.719,0.719
1,13,2,1.755,0.878,1.780,0.890
2,13,3,2.637,0.879,2.672,0.891
3,13,6,5.167,0.861,5.253,0.875
4,13,9,6.345,0.705,6.499,0.722
5,13,12,7.376,0.615,7.581,0.632
6,14,1,0.642,0.642,0.697,0.697
7,14,2,1.792,0.896,1.812,0.906
8,14,3,2.707,0.902,2.743,0.914
9,14,6,5.315,0.886,5.387,0.898


In [21]:
res1 = res.pivot(index="Threads", columns="# Cidades", values=["Speedup instância", "Eficiência instância"])
res2 = res.pivot(index="Threads", columns="# Cidades", values=["Speedup tempo total", "Eficiência tempo total"])
res1

Speedup instância             Eficiência instância            
# Cidades                13    14    15                   13    14    15
Threads                                                                 
1                     0.703 0.642 0.683                0.703 0.642 0.683
2                     1.755 1.792 1.814                0.878 0.896 0.907
3                     2.637 2.707 2.756                0.879 0.902 0.919
6                     5.167 5.315 5.429                0.861 0.886 0.905
9                     6.345 6.493 6.574                0.705 0.721 0.730
12                    7.376 7.526 7.533                0.615 0.627 0.628

In [22]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [23]:
res1

In [24]:
res2

Compare time dedicated to the sequential portion of the algorithms with total time

In [10]:
OUTPUT_EXP_DIR="output_experimental"
OUTPUT_THE_DIR="output_theoretical"
CITIES = [15]
THREADS = [6]
ITERATIONS = 10
EXECUTIONS = 50

In [16]:
for city in list(CITIES):
    sum_experimental, sum_theoretical = [], []
    for iteration in range(1,ITERATIONS + 1):
        file_name = OUTPUT_EXP_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        sum, instance_array = extract_time_stats_from_file(file_name)
        sum_experimental.append(sum)
        
        file_name = OUTPUT_THE_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        sum, instance_array = extract_time_stats_from_file(file_name)
        sum_theoretical.append(sum)
    print("Sequential portion analysis - Sequential {} cities: {} ([0, 1])".format(city, np.mean(sum_theoretical)/np.mean(sum_experimental)))

            
    for thread in list(THREADS):
        sum_experimental, sum_theoretical = [], []
        for iteration in range(1,ITERATIONS + 1):
            file_name = OUTPUT_EXP_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(thread) + "_threads_result.out"
            sum, instance_array = extract_time_stats_from_file(file_name)
            sum_experimental.append(sum)
            
            file_name = OUTPUT_THE_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(thread) + "_threads_result.out"
            sum, instance_array = extract_time_stats_from_file(file_name)
            sum_theoretical.append(sum)
        print("Sequential portion analysis - Parallel {} cities {} threads: {} ([0, 1])".format(city, thread, np.mean(sum_theoretical)/np.mean(sum_experimental)))

        

Sequential portion analysis - Sequential 15 cities: 8.36819776255044e-07 ([0, 1])
Sequential portion analysis - Parallel 15 cities 6 threads: 5.4934597843094955e-06 ([0, 1])
